#### Генерируем еще одну часть датасета для обучения BERT (для кода, который в репозитории preposition-generation)

В этом коде моя задача была научить программу правильно склонять личность на русском языке.

Есть начачало набор тегов **('играть', ' гитара,' фронтенд-разработчик')**, есть начало предложения, правильно собранное из этих тегов: **поиграть на гитаре в парке**

**Цель:** правильно подставить в это предложение последнее слово из набора тегов (это профессия человека) (**"поиграть на гитаре c фронтенд-разработчиком"**)

**ПРАВИЛА:**
- если через -, то склоняется только 2 слово 
- если предлог, то только первое слово 
- если сущ + прилаг, то оба слова 

In [1]:
import pandas as pd

# data = pd.read_excel('tags_with_professions.xlsx')
data = pd.read_excel('old_tags_with_new_professions.xlsx')
data

,tags,text_without_prof
0,"['играть', ' гитара,' фронтенд-разработчик']",поиграть на гитаре в парке
1,"['рисовать', ' граффити,' тестировщик программ...",нарисовать граффити на стене
2,"['посещать', ' пещера,' таргетолог']",посетить пещеры на каяке
3,"['сапсерфить', ' морской лев,' специалист по к...",посапсерфить с морскими львами
4,"['научиться танцевать', ' бачата,' системный а...",научиться танцевать бачату
...,...,...
403,"['Научиться играть', ' блэкджек,' инженер-испы...",Научиться играть в блэкдже в казино с Гарри Ки...
404,"['Поиграть', ' баккара,' биоэколог']",Поиграть в баккару со звукоинженером
405,"['Пофотографировать', ' бинго,' врач-терапевт']",Пофотографировать бинго в свинарнике
406,"['Порубиться', ' крэпс,' лаборант химического ...",Порубиться в крэпс в подвале МВД


In [2]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [3]:
def cut_part_text(tags,text):
    len_tags = len(tags.strip('[ ]').split(',')[0].strip("' '").split()) + len(tags.strip('[ ]').split(',')[1].strip("' '").split())
#     print(len_tags)

    list_of_words = text.split()
    if list_of_words[1] in ['на','с', 'в','за','по']:
        cut_text = ' '.join(text.split()[:len_tags+1])
    else:
        cut_text = ' '.join(text.split()[:len_tags])
    return cut_text

def change_person(person):
    
    if len(person.split()) == 2:
        
        if (('ADJF' in morph.parse(person.split()[0])[0].tag) and ('NOUN' in morph.parse(person.split()[1])[0].tag)) \
            or (('NOUN' in morph.parse(person.split()[0])[0].tag) and ('ADJF' in morph.parse(person.split()[1])[0].tag)):
            
            word1 = morph.parse(person.split()[0])[0]
            word2 = morph.parse(person.split()[1])[0]
            try:
                return  word1.inflect({"ablt"}).word + ' ' + word2.inflect({"ablt"}).word 
            except:
                print('BAD')
                return   word1.word +  ' ' + word2.word
        else:
            word = morph.parse(person.split()[0])[0]
            try:
                return  word.inflect({"ablt"}).word + ' ' +  ' '.join(person.split()[1:])
            except:
                print('BAD')
                return   word.word +  ' ' + ' '.join(person.split()[1:])
            
        
    elif len(person.split()) > 1:
        
        word = morph.parse(person.split()[0])[0]
        try:
            return  word.inflect({"ablt"}).word + ' ' +  ' '.join(person.split()[1:])
        except:
            print('BAD')
            return   word.word +  ' ' + ' '.join(person.split()[1:])
        
    elif len(person.split('-')) > 1:
        word = morph.parse(person.split('-')[1])[0]
        try:
            return person.split('-')[0] + '-' + word.inflect({"ablt"}).word
        except:
            print('BAD')
            return  person.split('-')[0] + '-' + word.word
        
    else:
        word = morph.parse(person)[0]
#     print(word)
    
        try:
            return word.inflect({"ablt"}).word
        except:
            print('BAD')
            return word.word


inputs = []
outputs = []
for i in data.index:
    tags = data['tags'].iloc[i]
    text = data['text_without_prof'].iloc[i]
    print(tags)
#     правила для склонения предмета 
    person = tags.strip('[ ]').split(',')[2].strip("' '")
    ch_person = change_person(person)
    cut_text = cut_part_text(tags,text)
    if ((ch_person[0] in  ['с', 'з', 'ш', 'ж']) and (ch_person[1] not in ['а', 'у', 'о', 'ы', 'и', 'э', 'я', 'ю', 'ё', 'е']))\
        or(ch_person[0] == 'щ') :
       
        norm_phrase = cut_text+' cо ' + ch_person
        
    else:
        norm_phrase = cut_text+' c ' + ch_person
    print(norm_phrase)
    
    inputs.append(tags)
    outputs.append(norm_phrase)
    

    


['играть', ' гитара,' фронтенд-разработчик']
поиграть на гитаре c фронтенд-разработчиком
['рисовать', ' граффити,' тестировщик программного обеспечения']
нарисовать граффити c тестировщиком программного обеспечения
['посещать', ' пещера,' таргетолог']
посетить пещеры c таргетологом
['сапсерфить', ' морской лев,' специалист по контекстной рекламе']
посапсерфить с морскими львами cо специалистом по контекстной рекламе
['научиться танцевать', ' бачата,' системный аналитик']
научиться танцевать бачату c системным аналитиком
['дружить', ' ламы,' системный администратор']
подружиться с ламами c системным администратором
['гонять', ' феррари,' программист']
погонять на феррари c программистом
['летать', ' вертолет,' оператор эвм']
полетать на вертолете c оператором эвм
['купаться', ' источник,' оператор пк']
искупаться в источнике c оператором пк
['оседлать', ' мустанг,' менеджер интернет-проектов']
оседлать мустанга c менеджером интернет-проектов
['учиться', ' серфинг,' контент-менеджер']
уч

Половить ящерицу c врач-педиатром
['Поизучать', ' черепаха,' врач-анестезиолог']
Поизучать черепах c врач-анестезиологом
['Пофотографироваться', ' змея,' врач']
Пофотографироваться со c врачом
['Погладить', ' полоз,' венеролог']
Погладить полоза c венерологом
['Покормить', ' круглоголовка,' акушер-гинеколог']
Покормить круглоголовку c акушер-гинекологом
['Обучить командам', ' собака,' финансовый директор']
Обучить командам собаку c финансовым директором
['Подстричь', ' кошка,' рекрутер']
Подстричь кошку c рекрутером
['Погладить', ' крыса,' менеджер проектов']
Погладить крысу c менеджером проектов
['Почесать', ' шиншилла,' менеджер по туризму']
Почесать шиншиллу c менеджером по туризму
['Одеть', ' морская свинка,' менеджер по работе с клиентами']
Одеть морскую свинку c менеджером по работе с клиентами
['Поохотиться с фоторужьем', ' медведь,' менеджер по персоналу']
Поохотиться с фоторужьем на медведя c менеджером по персоналу
['Погладить', ' леопард,' менеджер']
Погладить леопарда c мен

Пофотографировать виолончель cо связанными с машиностроением
['Научиться играть', ' гитара,' сварщик']
Научиться играть на cо сварщиком
['Поиграть', ' арфа,' сантехник']
Поиграть на арфе c сантехником
['Покрасить', ' контрабас,' проходчик']
Покрасить контрабас c проходчиком
['Исполнить соло', ' флейта,' профессии, связанные с электричеством']
Исполнить соло на c профессией
['Подудеть', ' валторна,' профессии на заводе']
Подудеть на валторне c профессией на заводе
['Послушать', ' тромбон,' обработка металла']
Послушать тромбон c обработкой металла
['Сбросить', ' труба,' нанотехнолог']
Сбросить трубу c нанотехнологом
['Научиться играть', ' трембита,' наладчик']
Научиться играть на c наладчиком
['Пофотографировать', ' кларнет,' монтажник']
Пофотографировать кларнет c монтажником
['Разобрать', ' саксофон,' механик']
Разобрать саксофон c механиком
['Послушать', ' фагот,' метролог']
Послушать фагот c метрологом
['Исполнить соло', ' гобой,' металлург']
Исполнить соло на c металлургом
['Научит

In [5]:
inputs = [i.replace("'","") for i in inputs]
data_in = pd.Series(inputs)
data_out = pd.Series(outputs)

data_res = pd.DataFrame(zip(data_in,data_out))

data_res = data_res.rename({0: 'inputs', 1: 'outputs'}, axis=1)

data_res


,inputs,outputs
0,"[играть, гитара, фронтенд-разработчик]",поиграть на гитаре c фронтенд-разработчиком
1,"[рисовать, граффити, тестировщик программного...",нарисовать граффити c тестировщиком программно...
2,"[посещать, пещера, таргетолог]",посетить пещеры c таргетологом
3,"[сапсерфить, морской лев, специалист по конте...",посапсерфить с морскими львами cо специалистом...
4,"[научиться танцевать, бачата, системный анали...",научиться танцевать бачату c системным аналитиком
...,...,...
403,"[Научиться играть, блэкджек, инженер-испытатель]",Научиться играть в c инженер-испытателем
404,"[Поиграть, баккара, биоэколог]",Поиграть в баккару c биоэкологом
405,"[Пофотографировать, бинго, врач-терапевт]",Пофотографировать бинго c врач-терапевтом
406,"[Порубиться, крэпс, лаборант химического анал...",Порубиться в крэпс c лаборантом химического ан...


In [6]:
data_res.to_excel('grammar_outputs_profession_based_on_existing_tags.xlsx', sheet_name = 'data', index=False)